this code is for making individual size distribution for specific cloud passes in one day of data 
* uses FAAM defined times for each cloud pass

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os

In [2]:
month = 'jul'
day = '30'
flight = 'c307'

In [ ]:
## getting faam files with breakdown of each flight - more info in faam_data.ipynb
faam_loc = '/badc/faam/data/2022/' ## this contains all faam flights - more than just DCMEX
file_list = glob(faam_loc+'*/') # whole string of path + filenames in location
file_names = [os.path.basename(os.path.dirname(file_path)) for file_path in file_list] # extract just directory names

## list just DCMEX flight:
start_file = "c298"
end_file = "c316"

selected_files = [f for f in file_names if f >= start_file and f <= end_file]
csv = glob(faam_loc + f'{flight}-{month}-{day}/' +'*.csv') # get the csv file containing flight summary for that day

In [ ]:
pd = pd.read_csv(csv[0])

* list out runs
* select one of the runs to plot - start + end time dictate 2D-s data look at

In [ ]:
## functions
## this function gets important bits from the netcdf -> (correct) time + index (particle diameter)
def important_bits(nc):
    time_var = nc.variables['time'][:]
    epoch_start = datetime.datetime(1970, 1, 1)
    datetime_values = [epoch_start + datetime.timedelta(seconds=float(time)) for time in time_var]

    index = nc.variables['ambient_particle_diameter'][:]
    return datetime_values, index

## function to extract specif variable (within ch0 / ch1 group) -> its own xr DataArray
## variable should be 2d - with time + index
def get_group_var(nc,ch_n,var_name,time_lst,idx_lst):
    var = nc[f'ch{ch_n}'][var_name]
    
    var_nc = xr.DataArray(var[:,:],
                          dims=['time', 'index'],  # Define dimensions
                          coords={'time': time_lst, 'index': idx_lst},  # Specify coords
                          attrs={key: var.getncattr(key) for key in var.ncattrs()})  # Copy attributes

    return var_nc



In [3]:
## flight summary where runs are
df = pd.read_csv('../other-measurements/flight-sum_faam_20220730_r0_c307.csv') ## whole csv
df.set_index('Event', inplace=True) # set the event as index for df
run_df = df[df.index.str.contains('Run')] # df with only cloud passes info

In [ ]:
path = '/gws/nopw/j04/dcmex/users/glloyd/'
nc_name = '2DS_MAN_20220730_v001_r0_c307.nc' #2DS_MAN_YYYYMMDD_v001_r0_name
nc_file = nc.Dataset(path+nc_name,'r')

time, index = important_bits(nc_file)

## things of interest: ambient_*I_particle_number_per_channel, for HI / MI / LI in ch0 and ch1
hi_0 = get_group_var(nc_file,0,'ambient_HI_particle_number_per_channel',time, index)
hi_1 = get_group_var(nc_file,1,'ambient_HI_particle_number_per_channel',time, index)

mi_0 = get_group_var(nc_file,0,'ambient_MI_particle_number_per_channel',time, index)
mi_1 = get_group_var(nc_file,1,'ambient_MI_particle_number_per_channel',time, index)

li_0 = get_group_var(nc_file,0,'ambient_LI_particle_number_per_channel',time, index)
li_1 = get_group_var(nc_file,1,'ambient_LI_particle_number_per_channel',time, index)

hi_av = (hi_0 + hi_1)/2
mi_av = (mi_0 + mi_1)/2
li_av = (li_0 + li_1)/2